# Análisis de Datos (II): práctica de merge, groupby y pivot_table

En este trabajo sobre pandas vamos a poner en práctica todos los conocimientos sobre pandas con el objetivo de ganar experiencia en el análisis de datos. 

Para ello disponemos de un dataset sobre valoración de películas, extraído de la web en la década de los 90 y 2000. El dataset consta de 943 usuarios, 1682 películas evaluadas y un total de 100.000 valoraciones. Todos los datos pueden encontrarse en 3 ficheros csv
- u.user: contiene la información de las usuarios
- u.item: contiene la información de las películas
- u.data: contiene la información de las valoraciones de los usuarios a las películas



In [1]:
import pandas as pd
import matplotlib.pyplot as mpl
import numpy as np

In [3]:
user = pd.read_csv('u.user',sep='|',header=None, names=['user_id','age','sex','occupation','zip_code'], encoding='utf-8')
item = pd.read_csv('u.item',sep='|',header=None,usecols=range(5),names=['movie_id','title','release_data','video_release','imdb_url'],encoding='utf-8')
data = pd.read_csv('u.data',sep='\t',names=['user_id','movie_id','rating','unix_timestamp'],encoding='utf-8')

Echa un ojo a las primeras filas de cada DataFrame para ver qué tipo de información contiene. Vamos a obtener algunas estadísticas iniciales de los dataframes

In [4]:
user.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


Muestra por pantalla el número de usuarios masculinos (sex=='M') y femeninos (sex=='F'). ¿Está balanceado?

In [7]:
user[user.sex=='M'].count()['user_id']

670

In [8]:
user[user.sex=='F'].count()['user_id']

273

In [9]:
user.groupby('sex')['user_id'].count()

sex
F    273
M    670
Name: user_id, dtype: int64

Obtén una descripción completa (función describe) de la columna rating de data. ¿Qué tipo de valoración se puede hacer en imdb? ¿Hay más comentarios positivos o negativos?

In [12]:
data.rating.describe()

count    100000.000000
mean          3.529860
std           1.125674
min           1.000000
25%           3.000000
50%           4.000000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

¿Has observado que podemos establecer una relación entre los 3 dataframes? Vamos a verlo: el primero (user) contiene la información de los usuarios que hacen los reviews. Parece ser que el user_id sirve para identificar a cada usuario unívocamente; El segundo (item) contiene la información de las películas que están siendo valoradas. Parece ser que movie_id sirve para identificar unívocamente cada una de las películas; El tercero (data) contiene, por cada fila, el identificador del usuario, el identificador de la película y la valoración numérica que dicho usuario ha otorgado a la correspondiente película. Está claro que podemos establecer una relación user-data-movie.

Combina los 3 dataframes en un dataframe llamado total que contenga, por cada fila, toda la información del usuario, toda la inforamción de la película y el rating que dicho usuario proporciona a dicha película

In [13]:
total = pd.merge(user,pd.merge(item,data))
total.head()

,user_id,age,sex,occupation,zip_code,movie_id,title,release_data,video_release,imdb_url,rating,unix_timestamp
0,1,24,M,technician,85711,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,5,874965758
1,1,24,M,technician,85711,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,3,876893171
2,1,24,M,technician,85711,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,4,878542960
3,1,24,M,technician,85711,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,3,876893119
4,1,24,M,technician,85711,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),3,889751712


¿Tienen todas las películas el mismo número de valoraciones? Crea una Series llamada numero_valoraciones que obtenga, para cada título de película, el número de valoraciones realizadas. Muestra las 10 películas con mayor número de valoraciones y las 10 con menos número de valoraciones.

In [14]:
total.groupby('title')['user_id'].count().sort_values(ascending=False).head(10)

title
Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
Name: user_id, dtype: int64

In [15]:
total.groupby('title')['user_id'].count().sort_values().head(10)

title
Á köldum klaka (Cold Fever) (1994)            1
Mille bolle blu (1993)                        1
Mat' i syn (1997)                             1
Marlene Dietrich: Shadow and Light (1996)     1
Man from Down Under, The (1943)               1
Mamma Roma (1962)                             1
Mad Dog Time (1996)                           1
Big Bang Theory, The (1994)                   1
MURDER and murder (1996)                      1
Coldblooded (1995)                            1
Name: user_id, dtype: int64

¿Cuáles son las películas con mayor valoración? Crea una Series valoracion_media que muestre por cada película la media de los ratings.

Muestra las 10 películas más valoradas

In [16]:
valoracion_media = total.groupby('title')['rating'].mean()
valoracion_media.sort_values(ascending=False).head(10)

title
Marlene Dietrich: Shadow and Light (1996)            5.0
Prefontaine (1997)                                   5.0
Santa with Muscles (1996)                            5.0
Star Kid (1997)                                      5.0
Someone Else's America (1995)                        5.0
Entertaining Angels: The Dorothy Day Story (1996)    5.0
Saint of Fort Washington, The (1993)                 5.0
Great Day in Harlem, A (1994)                        5.0
They Made Me a Criminal (1939)                       5.0
Aiqing wansui (1994)                                 5.0
Name: rating, dtype: float64

Observa que muchas de las películas no son muy conocidas. Eso es porque probablemente tengan muy pocas valoraciones. Para corregir esto, obtén la valoración media de aquellas películas que hayan recibido al menos 100 valoraciones. Ahora, ¿cuáles son las 10 películas más valoradas? ¿Y las menos?

In [19]:
numero_valoraciones = total.groupby('title')['rating'].count()
numero_valoraciones

title
'Til There Was You (1997)                  9
1-900 (1994)                               5
101 Dalmatians (1996)                    109
12 Angry Men (1957)                      125
187 (1997)                                41
                                        ... 
Young Guns II (1990)                      44
Young Poisoner's Handbook, The (1995)     41
Zeus and Roxanne (1997)                    6
unknown                                    9
Á köldum klaka (Cold Fever) (1994)         1
Name: rating, Length: 1664, dtype: int64

In [20]:
valoracion_media[numero_valoraciones>100].sort_values(ascending=False).head(10)

title
Close Shave, A (1995)               4.491071
Schindler's List (1993)             4.466443
Wrong Trousers, The (1993)          4.466102
Casablanca (1942)                   4.456790
Shawshank Redemption, The (1994)    4.445230
Rear Window (1954)                  4.387560
Usual Suspects, The (1995)          4.385768
Star Wars (1977)                    4.358491
12 Angry Men (1957)                 4.344000
Citizen Kane (1941)                 4.292929
Name: rating, dtype: float64

In [21]:
valoracion_media[numero_valoraciones>100].sort_values().head(10)

title
Cable Guy, The (1996)          2.339623
Jungle2Jungle (1997)           2.439394
Crash (1996)                   2.546875
Event Horizon (1997)           2.574803
Spawn (1997)                   2.615385
Batman Forever (1995)          2.666667
Batman Returns (1992)          2.683099
George of the Jungle (1997)    2.685185
Down Periscope (1996)          2.702970
Mimic (1997)                   2.742574
Name: rating, dtype: float64

A partir de ahora vamos a dejar en total únicamente aquellas películas con más de 100 valoraciones

In [23]:
total = total[total['title'].isin(numero_valoraciones[numero_valoraciones>100].index)]

Si tuvieses que recomendar 10 películas a un hombre o a una mujer, ¿eligirías las mismas? Vamos a comprobar si existen diferencias entre las películas más valoradas por hombres que por mujeres.

Para ello crea una pivot_table que nos muestre, por cada película, la valoración media hecha por mujeres y por hombres. Obtén las 10 más valoradas por mujeres y por hombres.

In [27]:
tabla = total.pivot_table(values='rating',index='title',columns='sex',aggfunc='mean')
tabla

sex,F,M
title,,
101 Dalmatians (1996),3.116279,2.772727
12 Angry Men (1957),4.269231,4.363636
2001: A Space Odyssey (1968),3.491228,4.103960
Absolute Power (1997),3.451613,3.343750
"Abyss, The (1989)",3.814815,3.540323
...,...,...
Willy Wonka and the Chocolate Factory (1971),3.752688,3.583691
"Wizard of Oz, The (1939)",4.159420,4.045198
"Wrong Trousers, The (1993)",4.444444,4.472527


In [28]:
tabla.sort_values(by='F',ascending=False).F.head(10)

title
Schindler's List (1993)             4.632911
Close Shave, A (1995)               4.631579
Shawshank Redemption, The (1994)    4.562500
Wrong Trousers, The (1993)          4.444444
Sling Blade (1996)                  4.411765
Secrets & Lies (1996)               4.407407
Casablanca (1942)                   4.400000
Much Ado About Nothing (1993)       4.381818
Good Will Hunting (1997)            4.355932
Rear Window (1954)                  4.351852
Name: F, dtype: float64

In [29]:
tabla.sort_values(by='M',ascending=False).M.head(10)

title
Casablanca (1942)                   4.473404
Wrong Trousers, The (1993)          4.472527
Close Shave, A (1995)               4.462366
Shawshank Redemption, The (1994)    4.410959
Schindler's List (1993)             4.406393
Rear Window (1954)                  4.400000
Usual Suspects, The (1995)          4.399061
Star Wars (1977)                    4.398148
12 Angry Men (1957)                 4.363636
Godfather, The (1972)               4.334416
Name: M, dtype: float64

¿Y si tuvieses que elegir aquellas películas que más diferencias presentan entre sexos? 
Para ello, añade una tercera fila a tabla llamada 'diferencia' en la que calcules la diferencia entre la valoración media de mujeres y de hombres. Muestra las 10 películas con mayor diferencia en favor de la valoración de las mujeres y las 10 con mayor diferencia en favor de los hombres.

In [30]:
tabla['diferencia'] = tabla.F-tabla.M
tabla

sex,F,M,diferencia
title,,,
101 Dalmatians (1996),3.116279,2.772727,0.343552
12 Angry Men (1957),4.269231,4.363636,-0.094406
2001: A Space Odyssey (1968),3.491228,4.103960,-0.612732
Absolute Power (1997),3.451613,3.343750,0.107863
"Abyss, The (1989)",3.814815,3.540323,0.274492
...,...,...,...
Willy Wonka and the Chocolate Factory (1971),3.752688,3.583691,0.168997
"Wizard of Oz, The (1939)",4.159420,4.045198,0.114223
"Wrong Trousers, The (1993)",4.444444,4.472527,-0.028083


In [31]:
tabla.sort_values(by='diferencia', ascending=False).head(10)

sex,F,M,diferencia
title,,,
"First Wives Club, The (1996)",3.491525,2.742574,0.748951
My Fair Lady (1964),4.238095,3.602410,0.635686
Batman Forever (1995),3.173913,2.538462,0.635452
"Net, The (1995)",3.482759,2.857143,0.625616
Grease (1978),3.754386,3.141593,0.612793
"Sound of Music, The (1965)",4.160000,3.564626,0.595374
Michael (1996),3.674419,3.118421,0.555998
Scream 2 (1997),3.629630,3.075949,0.553680
Grumpier Old Men (1995),3.447368,2.900000,0.547368


In [32]:
tabla.sort_values(by='diferencia').head(10)

sex,F,M,diferencia
title,,,
"Good, The Bad and The Ugly, The (1966)",3.187500,3.950413,-0.762913
True Romance (1993),3.000000,3.727273,-0.727273
Reservoir Dogs (1992),3.433333,4.127119,-0.693785
Waterworld (1995),2.263158,2.927711,-0.664553
"Bridge on the River Kwai, The (1957)",3.685714,4.307692,-0.621978
2001: A Space Odyssey (1968),3.491228,4.103960,-0.612732
Patton (1970),3.476190,4.086957,-0.610766
Full Metal Jacket (1987),3.173913,3.761905,-0.587992
Beavis and Butt-head Do America (1996),2.310345,2.897638,-0.587293


Cuando una película es buena...¿Todo el mundo dice que es buena? ¿Y al revés? ¿Qué películas tienen la mayor variabiliad entre sus valoraciones? Obtén las 10 con mayores desviaciones.


In [33]:
total.groupby('title')['rating'].std().sort_values(ascending=False).head(10)

title
Natural Born Killers (1994)               1.327240
Nightmare on Elm Street, A (1984)         1.313385
Lost Highway (1997)                       1.303494
Crash (1996)                              1.297237
Beavis and Butt-head Do America (1996)    1.259850
Evita (1996)                              1.234051
Grease (1978)                             1.232027
Happy Gilmore (1996)                      1.223152
Kingpin (1996)                            1.216547
Ace Ventura: Pet Detective (1994)         1.215735
Name: rating, dtype: float64